# Project Title
### Data Engineering Capstone Project

#### Project Summary

In this project, I plan to extract data from 4 different sources to analyze US immigration. This analysis will used data such as city monthly average temperature, city demographics (age, gender...), immigration data and a corresponding table between airport codes and cities.

We have 4 different sources : 


1. **I94 Immigration Data**: This data comes from the US National Tourism and Trade Office. This dataset contains all data from foreign visitors to the United States in the past (Customs and Border Protection Form I-94 Arrival/Departure Record or Form I-94W Nonimmigrant Visa Waiver Arrival/Departure Record).</p>

2. **World Temperature Data**: This dataset came from Kaggle. It contains average weather temperatures by city from 1743 to 2013. </p>

3. **U.S. City Demographic Data**: This data comes from OpenSoft. It contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65 000. This data comes from the US Census Bureau's 2015 American Community Survey.</p>

4. **Airport Code Table**: This is a simple table of airport codes and corresponding cities. </p>


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
    * Identify and gather the data you'll be using for your project (at least two sources and more than 1 million rows). See Project Resources for ideas of what data you can use.
    * Explain what end use cases you'd like to prepare the data for (e.g., analytics table, app back-end, source-of-truth database, etc.)
* Step 2: Explore and Assess the Data
    * Explore the data to identify data quality issues, like missing values, duplicate data, etc.
    * Document steps necessary to clean the data
* Step 3: Define the Data Model
    * Map out the conceptual data model and explain why you chose that model
    * List the steps necessary to pipeline the data into the chosen data model
* Step 4: Run ETL to Model the Data
    * Create the data pipelines and the data model
    * Include a data dictionary
    * Run data quality checks to ensure the pipeline ran as expected
        * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
        * Unit tests for the scripts to ensure they are doing the right thing
        * Source/count checks to ensure completeness
* Step 5: Complete Project Write Up
    * What's the goal? What queries will you want to run? How would Spark or Airflow be incorporated? Why did you choose the model you chose?
    * Clearly state the rationale for the choice of tools and technologies for the project.
    * Document the steps of the process.
    * Propose how often the data should be updated and why.
    * Post your write-up and final data model in a GitHub repo.
    * Include a description of how you would approach the problem differently under the following scenarios:
        * If the data was increased by 100x.
        * If the pipelines were run on a daily basis by 7am.
        * If the database needed to be accessed by 100+ people.

In [1]:
# Do all imports and installs here
import pandas as pd
import os
import glob
import re
from datetime import datetime, timedelta
from pprint import pprint
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, udf, year, month, avg, round, dayofweek, weekofyear, isnull
from pyspark.sql.types import StringType, IntegerType
from IPython.display import Image

In [ ]:
# Create Spark session
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
In this project, I plan to extract data from 4 different sources to analyze US immigration. This analysis will used data such as city monthly average temperature, city demographics (age, gender...), immigration data and a corresponding table between airport codes and cities. 

#### Describe and Gather Data 

We have 4 different sources : 


1. **I94 Immigration Data**: This data comes from the US National Tourism and Trade Office. This dataset contains all data from foreign visitors to the United States in the past (Customs and Border Protection Form I-94 Arrival/Departure Record or Form I-94W Nonimmigrant Visa Waiver Arrival/Departure Record).</p>

2. **World Temperature Data**: This dataset came from Kaggle. It contains average weather temperatures by city from 1743 to 2013. </p>

3. **U.S. City Demographic Data**: This data comes from OpenSoft. It contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65 000. This data comes from the US Census Bureau's 2015 American Community Survey.</p>

4. **Airport Code Table**: This is a simple table of airport codes and corresponding cities. </p>

In [2]:
# Read a sample of immigration data
df_immigration = pd.read_csv('immigration_data_sample.csv')
df_immigration.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [ ]:
# Read immigration data
df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [3]:
# Read the temperature data
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperatures = pd.read_csv(fname)
df_temperatures.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [4]:
# Read the demographics data
df_demographics = pd.read_csv('us-cities-demographics.csv', delimiter = ';')
df_demographics.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [5]:
# Readthe airport code table data 
df_airport = pd.read_csv('airport-codes_csv.csv')
df_airport.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [ ]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

In [ ]:
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immigration = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [6]:
df_immigration.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


### Step 2: Explore and Assess the Data

Goal : Aggregation per year and city 

#### Temperatures

In [7]:
df_temperatures.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [8]:
df_temperatures.Country.nunique()

159

In [9]:
# 159 are available in this dataset
df_temperatures.groupby('Country').City.count().sort_values(ascending=False).head()

Country
India            1014906
China             827802
United States     687289
Brazil            475580
Russia            461234
Name: City, dtype: int64

In [10]:
# We only want US cities
df_temperatures_clean = df_temperatures[df_temperatures.Country == "United States"]
df_temperatures_clean.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47555,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
47556,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
47557,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
47558,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
47559,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W


In [11]:
# dt to date 
df_temperatures_clean['dt'] =  pd.to_datetime(df_temperatures_clean['dt'])
df_temperatures_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687289 entries, 47555 to 8439246
Data columns (total 7 columns):
dt                               687289 non-null datetime64[ns]
AverageTemperature               661524 non-null float64
AverageTemperatureUncertainty    661524 non-null float64
City                             687289 non-null object
Country                          687289 non-null object
Latitude                         687289 non-null object
Longitude                        687289 non-null object
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 41.9+ MB


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
# Data from 1743 to 2013
df_temperatures_clean.dt.describe()

count                  687289
unique                   3239
top       1970-01-01 00:00:00
freq                      257
first     1743-11-01 00:00:00
last      2013-09-01 00:00:00
Name: dt, dtype: object

In [13]:
# Filter only on the last year 
df_temperatures_clean['year'], df_temperatures_clean['month'] = df_temperatures_clean['dt'].apply(lambda x: x.year), df_temperatures_clean['dt'].apply(lambda x: x.month)
df_temperatures_clean = df_temperatures_clean[df_temperatures_clean.year == 2013]
df_temperatures_clean.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,year,month
49871,2013-01-01,6.320,0.267,Abilene,United States,32.95N,100.53W,2013,1
49872,2013-02-01,8.116,0.222,Abilene,United States,32.95N,100.53W,2013,2
49873,2013-03-01,12.503,0.273,Abilene,United States,32.95N,100.53W,2013,3
49874,2013-04-01,15.753,0.342,Abilene,United States,32.95N,100.53W,2013,4
49875,2013-05-01,22.545,0.125,Abilene,United States,32.95N,100.53W,2013,5


In [14]:
# 248 cities in the dataframe 
df_temperatures_clean.City.nunique()

248

In [15]:
df_temperatures_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2313 entries, 49871 to 8439246
Data columns (total 9 columns):
dt                               2313 non-null datetime64[ns]
AverageTemperature               2312 non-null float64
AverageTemperatureUncertainty    2312 non-null float64
City                             2313 non-null object
Country                          2313 non-null object
Latitude                         2313 non-null object
Longitude                        2313 non-null object
year                             2313 non-null int64
month                            2313 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 180.7+ KB


In [16]:
df_temperatures_clean[df_temperatures_clean.isna()]

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,year,month
49871,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49872,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49873,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49874,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49875,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49876,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49877,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49878,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49879,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140296,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_temperatures_clean.dropna(inplace=True)
df_temperatures_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2312 entries, 49871 to 8439246
Data columns (total 9 columns):
dt                               2312 non-null datetime64[ns]
AverageTemperature               2312 non-null float64
AverageTemperatureUncertainty    2312 non-null float64
City                             2312 non-null object
Country                          2312 non-null object
Latitude                         2312 non-null object
Longitude                        2312 non-null object
year                             2312 non-null int64
month                            2312 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 180.6+ KB


In [18]:
df_temperatures_clean.shape

(2312, 9)

In [19]:
df_temperatures_clean[df_temperatures_clean[['City','dt']].duplicated()].sort_values(by=['dt','City']).head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,year,month
406591,2013-01-01,1.842,0.181,Arlington,United States,39.38N,76.99W,2013,1
488248,2013-01-01,-2.539,0.285,Aurora,United States,40.99N,87.34W,2013,1
1736671,2013-01-01,0.536,0.257,Columbus,United States,39.38N,83.24W,2013,1
2593404,2013-01-01,8.813,0.651,Glendale,United States,34.56N,118.70W,2013,1
5718753,2013-01-01,8.813,0.651,Pasadena,United States,34.56N,118.70W,2013,1


In [20]:
df_temperatures_clean[(df_temperatures_clean.City == 'Arlington')&(df_temperatures_clean.dt == '2013-01-01')]

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,year,month
403352,2013-01-01,8.598,0.253,Arlington,United States,32.95N,96.70W,2013,1
406591,2013-01-01,1.842,0.181,Arlington,United States,39.38N,76.99W,2013,1


In [21]:
# The unique key for temperatures is City x dt, but there are some doublons because of various statements.
# 3% of the lines are doublons. 
100*df_temperatures_clean[df_temperatures_clean[['City','dt']].duplicated()].City.count()/len(df_temperatures_clean)

3.5034602076124566

In [22]:
# I will compute these lines with average temperature and average temperature uncertainty
df_temperatures_clean = df_temperatures_clean.groupby(['dt','City','Country','year','month']).mean().reset_index()

In [23]:
# There is no more duplicates lines.
df_temperatures_clean[df_temperatures_clean[['City','dt']].duplicated()].City.count()

0

##### Immigraton

In [24]:
df_immigration.columns

Index(['Unnamed: 0', 'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port',
       'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa',
       'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd',
       'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum',
       'airline', 'admnum', 'fltno', 'visatype'],
      dtype='object')

In [25]:
df_immigration.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [26]:
df_immigration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
Unnamed: 0    1000 non-null int64
cicid         1000 non-null float64
i94yr         1000 non-null float64
i94mon        1000 non-null float64
i94cit        1000 non-null float64
i94res        1000 non-null float64
i94port       1000 non-null object
arrdate       1000 non-null float64
i94mode       1000 non-null float64
i94addr       941 non-null object
depdate       951 non-null float64
i94bir        1000 non-null float64
i94visa       1000 non-null float64
count         1000 non-null float64
dtadfile      1000 non-null int64
visapost      382 non-null object
occup         4 non-null object
entdepa       1000 non-null object
entdepd       954 non-null object
entdepu       0 non-null float64
matflag       954 non-null object
biryear       1000 non-null float64
dtaddto       1000 non-null object
gender        859 non-null object
insnum        35 non-null float64
airline       967 non

In [27]:
# Drop columns with too many missing values
df_immigration_clean = df_immigration.drop(columns=['visapost','occup','entdepu','insnum'])

In [28]:
df_immigration_clean.cicid.nunique()

1000

In [29]:
df_immigration_clean.cicid.count()

1000

In [30]:
df_immigration_clean.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepa,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,G,O,M,1955.0,07202016,F,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,G,R,M,1990.0,10222016,M,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,G,O,M,1940.0,07052016,M,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,G,O,M,1991.0,10272016,M,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,Z,K,M,1997.0,07042016,F,NaN,4.232257e+10,LAND,WT


In [31]:
df_immigration_clean.columns

Index(['Unnamed: 0', 'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port',
       'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa',
       'count', 'dtadfile', 'entdepa', 'entdepd', 'matflag', 'biryear',
       'dtaddto', 'gender', 'airline', 'admnum', 'fltno', 'visatype'],
      dtype='object')

In [32]:
# The unique key for immigration is cicid
df_immigration_clean[df_immigration_clean['cicid'].duplicated()]

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepa,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype


##### Demographics

In [33]:
df_demographics.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [34]:
df_demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


In [35]:
# Looking for null data : null data for Male or Female population but also for number of veterans, foreign-born and average household size 
df_demographics_nulls = df_demographics[(df_demographics['Male Population'].isnull())|
                (df_demographics['Female Population'].isnull())|
                (df_demographics['Number of Veterans'].isnull())|
                (df_demographics['Foreign-born'].isnull())|
                (df_demographics['Average Household Size'].isnull())]
df_demographics_nulls

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
111,San Juan,Puerto Rico,41.4,155408.0,186829.0,342237,NaN,NaN,NaN,PR,Hispanic or Latino,335559
155,Caguas,Puerto Rico,40.4,34743.0,42265.0,77008,NaN,NaN,NaN,PR,Hispanic or Latino,76349
258,Carolina,Puerto Rico,42.0,64758.0,77308.0,142066,NaN,NaN,NaN,PR,American Indian and Alaska Native,12143
333,The Villages,Florida,70.5,NaN,NaN,72590,15231.0,4034.0,NaN,FL,Hispanic or Latino,1066
449,The Villages,Florida,70.5,NaN,NaN,72590,15231.0,4034.0,NaN,FL,Black or African-American,331
637,Carolina,Puerto Rico,42.0,64758.0,77308.0,142066,NaN,NaN,NaN,PR,Hispanic or Latino,139967
1437,The Villages,Florida,70.5,NaN,NaN,72590,15231.0,4034.0,NaN,FL,White,72211
1747,San Juan,Puerto Rico,41.4,155408.0,186829.0,342237,NaN,NaN,NaN,PR,American Indian and Alaska Native,4031
1748,Mayagüez,Puerto Rico,38.1,30799.0,35782.0,66581,NaN,NaN,NaN,PR,Asian,235
1995,Ponce,Puerto Rico,40.5,56968.0,64615.0,121583,NaN,NaN,NaN,PR,Hispanic or Latino,120705


In [36]:
# There is 16 lines with missing values, which represents only 0,55% of the number of lines
100*df_demographics_nulls.City.count()/df_demographics.City.count()

0.55344171566931855

In [37]:
# I can drop these columns
df_demographics_clean = df_demographics[(df_demographics['Male Population'].notnull())&
                (df_demographics['Female Population'].notnull())&
                (df_demographics['Number of Veterans'].notnull())&
                (df_demographics['Foreign-born'].notnull())&
                (df_demographics['Average Household Size'].notnull())]

In [38]:
# Update format 
df_demographics_clean['Male Population'] = df_demographics_clean['Male Population'].astype(int)
df_demographics_clean['Female Population'] = df_demographics_clean['Female Population'].astype(int)
df_demographics_clean['Number of Veterans'] = df_demographics_clean['Number of Veterans'].astype(int)
df_demographics_clean['Foreign-born'] = df_demographics_clean['Foreign-born'].astype(int)
df_demographics_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2875 entries, 0 to 2890
Data columns (total 12 columns):
City                      2875 non-null object
State                     2875 non-null object
Median Age                2875 non-null float64
Male Population           2875 non-null int64
Female Population         2875 non-null int64
Total Population          2875 non-null int64
Number of Veterans        2875 non-null int64
Foreign-born              2875 non-null int64
Average Household Size    2875 non-null float64
State Code                2875 non-null object
Race                      2875 non-null object
Count                     2875 non-null int64
dtypes: float64(2), int64(6), object(4)
memory usage: 292.0+ KB


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [39]:
df_demographics_clean[df_demographics_clean[['City','Race']].duplicated()].sort_values(by=['City','Race']).head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
1470,Albany,New York,32.8,47627,50825,98452,3643,11948,2.08,NY,American Indian and Alaska Native,1611
2472,Albany,New York,32.8,47627,50825,98452,3643,11948,2.08,NY,Asian,8090
2278,Albany,New York,32.8,47627,50825,98452,3643,11948,2.08,NY,Black or African-American,31303
2552,Albany,Georgia,33.3,31695,39414,71109,5409,861,2.38,GA,Hispanic or Latino,1783
2000,Albany,New York,32.8,47627,50825,98452,3643,11948,2.08,NY,White,58368


In [40]:
df_demographics_clean[(df_demographics_clean.City == 'Albany')].sort_values(by='Race')

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
1165,Albany,Georgia,33.3,31695,39414,71109,5409,861,2.38,GA,American Indian and Alaska Native,445
1470,Albany,New York,32.8,47627,50825,98452,3643,11948,2.08,NY,American Indian and Alaska Native,1611
1616,Albany,Georgia,33.3,31695,39414,71109,5409,861,2.38,GA,Asian,650
2472,Albany,New York,32.8,47627,50825,98452,3643,11948,2.08,NY,Asian,8090
2050,Albany,Georgia,33.3,31695,39414,71109,5409,861,2.38,GA,Black or African-American,53440
2278,Albany,New York,32.8,47627,50825,98452,3643,11948,2.08,NY,Black or African-American,31303
1809,Albany,New York,32.8,47627,50825,98452,3643,11948,2.08,NY,Hispanic or Latino,9368
2552,Albany,Georgia,33.3,31695,39414,71109,5409,861,2.38,GA,Hispanic or Latino,1783
1260,Albany,Georgia,33.3,31695,39414,71109,5409,861,2.38,GA,White,17160
2000,Albany,New York,32.8,47627,50825,98452,3643,11948,2.08,NY,White,58368


In [41]:
# The unique key for demographics is City x State x Race
df_demographics_clean[df_demographics_clean[['City','State','Race']].duplicated()]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count


##### Airport

In [42]:
df_airport.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [43]:
df_airport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


In [44]:
df_airport.groupby('continent').count()

,ident,type,name,elevation_ft,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
continent,,,,,,,,,,,
AF,3362,3362,3362,2857,3115,3362,2876,2124,1071,61,3362
AN,28,28,28,26,28,28,16,24,1,10,28
AS,5350,5350,5350,3402,5350,5350,4527,2617,1726,620,5350
EU,7840,7840,7840,5629,7840,7840,6475,3872,1120,831,7840
OC,3067,3067,3067,1981,3067,3067,1512,2683,1316,709,3067
SA,7709,7709,7709,7314,7709,7709,7238,5761,988,2234,7709


In [45]:
df_airport.groupby('type').count()

,ident,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
type,,,,,,,,,,,
balloonport,24,24,21,5,24,24,23,18,0,20,24
closed,3606,3606,2614,1534,3602,3606,2968,744,279,499,3606
heliport,11287,11287,9545,4252,11287,11287,10927,8517,68,7593,11287
large_airport,627,627,602,418,626,627,611,601,602,202,627
medium_airport,4550,4550,4342,3303,4539,4550,4018,4279,3859,970,4550
seaplane_base,1016,1016,827,60,1016,1016,792,902,143,935,1016
small_airport,33965,33965,30118,17784,33734,33965,30060,25969,4238,18467,33965


In [46]:
df_airport_clean = df_airport

In [47]:
# The unique key for airport is ident
df_airport_clean[df_airport_clean['ident'].duplicated()]

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
The star schema is a good data model for our analysis with fact and dimension tables. 

![Data model](Data_model.png)

#### 3.2 Mapping Out Data Pipelines
List of the steps necessary to pipeline the data into the star schema: 

- Load the data from csv and sas files
- Clean the data (as detailed in part 2)
- Create dimension and fact tables 

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [48]:
# fct_immigration
df_fct_immigration = df_immigration_clean[['cicid','gender','visatype','airline','biryear','i94yr','i94mon','i94port','arrdate','i94mode','depdate','i94bir','i94visa','entdepa','dtaddto']]
df_fct_immigration.head()

,cicid,gender,visatype,airline,biryear,i94yr,i94mon,i94port,arrdate,i94mode,depdate,i94bir,i94visa,entdepa,dtaddto
0,4084316.0,F,WT,JL,1955.0,2016.0,4.0,HHW,20566.0,1.0,20573.0,61.0,2.0,G,07202016
1,4422636.0,M,B2,*GA,1990.0,2016.0,4.0,MCA,20567.0,1.0,20568.0,26.0,2.0,G,10222016
2,1195600.0,M,WT,LH,1940.0,2016.0,4.0,OGG,20551.0,1.0,20571.0,76.0,2.0,G,07052016
3,5291768.0,M,B2,QR,1991.0,2016.0,4.0,LOS,20572.0,1.0,20581.0,25.0,2.0,G,10272016
4,985523.0,F,WT,NaN,1997.0,2016.0,4.0,CHM,20550.0,3.0,20553.0,19.0,2.0,Z,07042016


In [49]:
# dim_temperature
df_temperatures_clean.rename(columns = {'City':'city','Country':'country','AverageTemperature':'average_temperature','AverageTemperatureUncertainty':'Average_temperature_uncertainty'}, inplace=True)
df_dim_temperature = df_temperatures_clean[['dt','city','country','average_temperature','Average_temperature_uncertainty']]
df_dim_temperature.reset_index(inplace=True,drop=True)
df_dim_temperature.reset_index(inplace=True)
df_dim_temperature.rename(columns= {'index':'id'},inplace=True)
df_dim_temperature.head()

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,id,dt,city,country,average_temperature,Average_temperature_uncertainty
0,0,2013-01-01,Abilene,United States,6.320,0.267
1,1,2013-01-01,Akron,United States,-1.086,0.220
2,2,2013-01-01,Albuquerque,United States,-1.981,0.371
3,3,2013-01-01,Alexandria,United States,1.842,0.181
4,4,2013-01-01,Allentown,United States,-0.968,0.290


In [50]:
# dim_demographic
df_demographics_clean.rename(columns = {'City':'city','State':'state','Median Age':'median_age','Male Population':'male_pop','Female Population':'female_pop','Total Population':'total_pop','Number of Veterans':'number_veterans','Foreign-born':'foreign_born','State Code':'state_code','Race':'race','Count':'count'}, inplace=True)
df_dim_demographic = df_demographics_clean[['city','race','state','median_age','male_pop','female_pop','total_pop','number_veterans','foreign_born','state_code']]
df_dim_demographic.reset_index(inplace=True,drop=True)
df_dim_demographic.reset_index(inplace=True)
df_dim_demographic.rename(columns= {'index':'id'},inplace=True)
df_dim_demographic.head()

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,id,city,race,state,median_age,male_pop,female_pop,total_pop,number_veterans,foreign_born,state_code
0,0,Silver Spring,Hispanic or Latino,Maryland,33.8,40601,41862,82463,1562,30908,MD
1,1,Quincy,White,Massachusetts,41.0,44129,49500,93629,4147,32935,MA
2,2,Hoover,Asian,Alabama,38.5,38040,46799,84839,4819,8229,AL
3,3,Rancho Cucamonga,Black or African-American,California,34.5,88127,87105,175232,5821,33878,CA
4,4,Newark,White,New Jersey,34.6,138040,143873,281913,5829,86253,NJ


In [51]:
# dim_airport
df_dim_airport = df_airport_clean[['ident','type','name','iso_country','iso_region','municipality']]
df_dim_airport.head()

,ident,type,name,iso_country,iso_region,municipality
0,00A,heliport,Total Rf Heliport,US,US-PA,Bensalem
1,00AA,small_airport,Aero B Ranch Airport,US,US-KS,Leoti
2,00AK,small_airport,Lowell Field,US,US-AK,Anchor Point
3,00AL,small_airport,Epps Airpark,US,US-AL,Harvest
4,00AR,closed,Newport Hospital & Clinic Heliport,US,US-AR,Newport


#### 4.2 Data Quality Checks
Various tests fort check data quality sush as: 
- if the table exists or not
- if the table is empty or not 
- if there is a unique key for each table

In [52]:
def table_exists(df):
    if df is not None:
        return True
    else:
        return False
        
if table_exists(df_fct_immigration) & table_exists(df_dim_temperature) & table_exists(df_dim_demographic) & table_exists(df_dim_airport):
    print("data quality check passed: all tables exist.")
else:
    print("Data quality check failed: some tables are missing.")

data quality check passed: all tables exist.


In [53]:
def table_not_empty(df):
    return not df.empty

if table_not_empty(df_fct_immigration) & table_not_empty(df_dim_temperature) & table_not_empty(df_dim_demographic) & table_not_empty(df_dim_airport):
    print("data quality check passed: all tables are not empty.")
else:
    print("data quality check faile: some tables are empty.")

data quality check passed: all tables are not empty.


In [54]:
def table_unique_id(df,unique_id):
    return df.unique_id.nunique() == len(df)

if df_fct_immigration.cicid.nunique() == len(df_fct_immigration) and df_dim_temperature.id.nunique() == len(df_dim_temperature) and df_dim_demographic.id.nunique() == len(df_dim_demographic) and df_dim_airport.ident.nunique() == len(df_dim_airport) : 
    print("data quality check passed: all tables have a unique key.")
else:
    print("data quality check faile: some tables do not have a unique key.")

data quality check passed: all tables have a unique key.


#### 4.3 Data dictionary 

![Data dictionary immigration](Data_dictionary_immigration.png)

![Data dictionary demographic](Data_dictionary_demographic.png)

![Data dictionary temperature](Data_dictionary_temperature.png)

![Data dictionary airport](Data_dictionary_airport.png)

#### Step 5: Complete Project Write Up

* There are a large amount of data from various file with different formats. 
  A good technology for this kind of situation is to use Spark. 
  
* This analysis could be updated every month, as all data sources is updated monthly. 

* If the data was increaed by 100x, it would be necessary to increase the number of nodes in the cluster for using Spark. 
* If the data populates a dashboard that must be updated on a daily basis by 7am every day, we could use an ETL, as Apache Airflow, which could schedule, run the data pipelines and test results as wanted. 
* If the database need to be accessed by 100+ people, it would be better to use a data warehouse (such as Amazon Redshift) to share the information with everyone. 